### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [151]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd

## Carga de datos

In [152]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [153]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [154]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [155]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [156]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [157]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [158]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [159]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [160]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [161]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 481
print(newsgroups_train.data[idx])


Probably would not be fatal in an adult at that dose, but could kill
a child.  Patient would be very somnolent, with dilated pupils, low
blood pressure.  Possibly cardiac arrhythmias.  


-- 
----------------------------------------------------------------------------
Gordon Banks  N3JXP      | "Skepticism is the chastity of the intellect, and
geb@cadre.dsl.pitt.edu   |  it is shameful to surrender it too soon." 


In [162]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [163]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.56537615, 0.55605743, ..., 0.        , 0.        ,
       0.        ])

In [164]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 481, 8550, 8660, ..., 4315, 4314, 1335], dtype=int64)

In [165]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [166]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'sci.med'

In [167]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

sci.med
sci.med
sci.med
sci.med
sci.med


### Modelo de clasificación Naïve Bayes

In [168]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [169]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [170]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

# Desafío 1

## Consigna

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


## Resolución

## **1**. Vectorizar documentos. 

Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.


Obtención del conjunto de datos

In [171]:
# Obtención de documetos del dataset, separación en train y test
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# Obtengo labels del dataset, para poder saber a qué categoría pertenece cada documento
y_train = newsgroups_train.target

Vectorización
- Utilizando TF-IDF

In [172]:
# Vectorización usando TF-IDF
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [173]:
# Diccionario para ir de indices a palabras
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

Funciones auxiliares:
- crop_text: Corta el texto a un número especificado de caracteres. Se utilizará para mejorar la visualización el notebook.
- get_similar_documents: Obtiene los N documentos más similares al documento que se indique, retornando los índices de los mismos y sus categorías

In [174]:
def crop_text(text, max_length=200):
    """Función auxiliar para cortar texto y mejorar visualización del notebook."""
    if len(text) > max_length:
        return text[:max_length - 3] + "..."
    return text

def get_similar_documents(X_train,newsgroups_train,y_train,document_idx,N=5):  
    """Función para obtener los N documentos más similares al actual."""  
    cossim = cosine_similarity(X_train[document_idx], X_train)[0]
    mas_similares_idxs = np.argsort(cossim)[::-1][1:(N+1)]
    mas_similares_cat = [newsgroups_train.target_names[y_train[i]] for i in mas_similares_idxs]
    return mas_similares_idxs,mas_similares_cat

Obtención de documentos similares:
- Se eligen 5 documentos al azar.
- Se imprime el contenido reducido (a máximo 200 caracteres) de cada documento.
- Se imprime información sobre su categoría.
- Se imprimen indices de los 5 más similares y sus categorías.

In [266]:
# Variables de configuración
cant_documentos = 5
limite_texto = 200 # 200 caracteres
# random_idxs = np.random.randint(0,X_train.shape[0],size=cant_documentos) #Descomentar para obtener documentos al azar
random_idxs = [2870,10213,1282,10470,4234] # Se fijan los valores para que permanezcan al volver ejecutar el notebook
print(f"Documentos tomados al azar (indices): {random_idxs}")
dict_similares = {}

# Ciclo for para imprimir detalles del documento y obtener sus 5 más similares
for i,idx in enumerate(random_idxs):
    # Obtengo los 5 documentos más similares utilizando similitud coseno
    mostsim_idx,mostsim_cat = get_similar_documents(X_train,newsgroups_train,y_train,idx,N=5)
    dict_similares[f"{idx}"] = {
        "idx": mostsim_idx,
        "cat": mostsim_cat
    }
    print("\n")
    print("-"*80)
    print(f"***** DOCUMENTO {i}. idx={idx}. Categoria: {newsgroups_train.target_names[y_train[idx]]} **********************")
    print(f"***** Similares (indices): {mostsim_idx}")
    print(f"***** Similares (categoría): {mostsim_cat}")
    print("-"*80)
    contenido_documento = newsgroups_train.data[idx]
     # Se limitan los caracteres del texto ya que con pocas palabras se entiende de qué trata
     #  el documento y mejora mucho la visualización en el notebook.
    contenido_documento = crop_text(contenido_documento,limite_texto)
    print(contenido_documento)
    print("-"*80)

Documentos tomados al azar (indices): [ 2870 10213  1282 10470  4234]


--------------------------------------------------------------------------------
***** DOCUMENTO 0. idx=2870. Categoria: sci.med **********************
***** Similares (indices): [ 218 9056 8441 1593 8572]
***** Similares (categoría): ['sci.med', 'sci.med', 'sci.med', 'sci.med', 'comp.windows.x']
--------------------------------------------------------------------------------
=There is no contradiction here. It is essential in the sense that your
=body needs it. It is non-essential in the sense that your body can
=produce enough of it without supplement.

And when you'r...
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
***** DOCUMENTO 1. idx=10213. Categoria: rec.autos **********************
***** Similares (indices): [ 2697  1123   877  1777 11152]
***** Similares (categoría): ['rec.autos', 'rec.aut

- Puede observarse que los documentos similares pertenecen casi todos a la misma categoría que el documento principal en cuestión.
- Veamos ahora el texto de cada documento similar, para cada uno de los 5 documentos elegidos al azar:

### Documento 1

In [267]:
# Funcion auxiliar para imprimir más rápido detalles de documentos similares
def imprimir_similares(document):
    document_idx = random_idxs[document-1]
    print(f"El documento {document} pertenece a la categoría {newsgroups_train.target_names[y_train[document_idx]]}\n")
    for i,idx in enumerate(dict_similares[f"{document_idx}"]["idx"]):
        cat = dict_similares[f"{document_idx}"]["cat"][i]
        print("-"*80)
        print(f"** Texto del documento similar. Idx: {idx} Categoría: {newsgroups_train.target_names[y_train[idx]]}")
        print("-"*80)
        print(crop_text(newsgroups_train.data[idx],300))
        print("-"*80)
        print("\n")


In [268]:
imprimir_similares(document=1)

El documento 1 pertenece a la categoría sci.med

--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 218 Categoría: sci.med
--------------------------------------------------------------------------------

There is no contradiction here. It is essential in the sense that your
body needs it. It is non-essential in the sense that your body can
produce enough of it without supplement.
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 9056 Categoría: sci.med
--------------------------------------------------------------------------------
=
=> Now here is a new one: vomiting. My guess is that MSG becomes the number one
=> suspect of any problem. In this case. it might be just food poisoning. But
=> if you heard things about MSG, you may think it must be it.
=
=----------
=
=Yea

### Documento 2

In [269]:
imprimir_similares(document=2)

El documento 2 pertenece a la categoría rec.autos

--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 2697 Categoría: rec.autos
--------------------------------------------------------------------------------

You'll have fun looking for the rear-end gears on an SHO--the Taurus
is a front-wheel-drive vehicle...

I went back and checked the article again; The SHO Wagon is quicker
than the SHO automatic, but significantly slower than the REAL SHO.

Why anyone would order an SHO with an automatic transmi...
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 1123 Categoría: rec.autos
--------------------------------------------------------------------------------

	I was hoping for something like "The chassis exhibits X 
degrees of flex when subjected to forces of more than Y 

### Documento 3

In [270]:
imprimir_similares(document=3)

El documento 3 pertenece a la categoría rec.sport.baseball

--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 1606 Categoría: rec.sport.baseball
--------------------------------------------------------------------------------



It's not a cliche, and (unlike your comments below) it's not a tautology.
It needn't have been true.  If every pitcher in baseball were essentially
the same in quality (i.e. if the variance of pitching ability were much
smaller than the variance of batting ability), then scoring runs would be
...
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 6032 Categoría: rec.sport.baseball
--------------------------------------------------------------------------------
The O's just lost to the Rangers a few minutes ago I was not too happy about
the pitchin

### Documento 4

In [271]:
imprimir_similares(document=4)

El documento 4 pertenece a la categoría talk.politics.guns

--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 4271 Categoría: talk.politics.misc
--------------------------------------------------------------------------------
THE WHITE HOUSE

                    Office of the Press Secretary
______________________________________________________________
For Immediate Release                             April 13, 1993     

	     
                      REMARKS BY THE PRESIDENT,
               SECRETARY OF EDUCATION RIC...
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 9623 Categoría: talk.politics.mideast
--------------------------------------------------------------------------------
Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012
                 

### Documento 5

In [272]:
imprimir_similares(document=5)

El documento 5 pertenece a la categoría comp.windows.x

--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 4614 Categoría: comp.windows.x
--------------------------------------------------------------------------------
Archive-name: x-faq/part5
Last-modified: 1993/04/04

----------------------------------------------------------------------
Subject: 119)  I'm writing a widget and can't use a float as a resource value.

Float resources are not portable; the size of the value may be larger than
the size of an XtP...
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
** Texto del documento similar. Idx: 7967 Categoría: comp.windows.x
--------------------------------------------------------------------------------
Archive-name: x-faq/part1
Last-modified: 1993/04/04

This article and several following contain the 

- Nuevamente se observa que no solo la categoría de los documentos obtenidos con similitud coseno coincide, sino que la temática es parecida.
- Por ejemplo, para el documento 5 todos los mails similares pertenecen a la categoría "comp.window" y se tratan sobre preguntas de software o del sistema operativo.

## **2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [292]:
# Obtención de datos del dataset
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# Función auxiliar para guardar métricas
def almacenar_metricas(clasificador,vectorizador,f1_score,existing_df=None):
    """Almacena métricas en un dataframe para facilitar su visualización."""
    df = pd.DataFrame({
                            "Clasificador":[clasificador],
                            "Vectorizador": [vectorizador],
                            "F1-score": [f1_score]
                            })
    if existing_df is None:
        return df

    df_ = pd.concat([existing_df,df],ignore_index=True)
    return df_

### 2.1 Naive Bayes MultinomialNB y vectorizador TF-IDF con parámetros por defecto

In [293]:
# Vectorización TF-IDF
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

# Clasificador: Naive Bayes Multinomial NB
clf = MultinomialNB()
clf.fit(X_train, y_train)
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)
score = f1_score(y_test, y_pred, average='macro')
score

0.5854345727938506

In [294]:
# Almacenamiento y muestra de métricas
df_metricas = almacenar_metricas(
        clasificador="MultinomialNB",
        vectorizador="TF-IDF con parámetros por defecto",
        f1_score=score
        )

display(df_metricas)

,Clasificador,Vectorizador,F1-score
0,MultinomialNB,TF-IDF con parámetros por defecto,0.585435


### 2.2 Naive Bayes MultinomialNB con parámetros por defecto y modificando parámetros de instanciación del vectorizador TF-IDF
- Para el vectorizador TF-IDF se modifican los parámetros:
    - min_df: Frecuencia mínima en la que debe aparecer un término para ser considerado. Se elige 0.1% , ya que se quieren eliminar términos muy raros que aparezcan una sola vez, pero podría haber términos que aparezcan con poca frecuencia y aporten mucha información. 
    - max_df: Frecuencia máxima de apareción de un término. Se elige 90% para eliminar del vocabulario artículos u otros elementos que no aportan información.
    - strip_accents: 'ascii' para remover todos los acentos y llevarlos a su caracter ASCII más cercano. Como el lenguaje de los documentos es inglés, este parámetro no debería afectar casi a ningún término.

In [295]:
# Vectorización TF-IDF
tfidfvect = TfidfVectorizer(min_df=(0.1/100),max_df=0.9,strip_accents='ascii')
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

# Clasificador: Naive Bayes Multinomial NB
clf = MultinomialNB()
clf.fit(X_train, y_train)
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)
score = f1_score(y_test, y_pred, average='macro')
score

0.6145475022611062

In [296]:
# Almacenamiento y muestra de métricas
df_metricas = almacenar_metricas(
        clasificador="MultinomialNB",
        vectorizador="TF-IDF con min_df=0.1%, max_df=90% y strip_accents='ascii'",
        f1_score=score,
        existing_df=df_metricas
        )

df_metricas

,Clasificador,Vectorizador,F1-score
0,MultinomialNB,TF-IDF con parámetros por defecto,0.585435
1,MultinomialNB,"TF-IDF con min_df=0.1%, max_df=90% y strip_acc...",0.614548


### 2.3 Idem 2.2 pero utilizando clasificador Naive Bayes ComplementNB

- Vectorizador: mismo que en punto 2.2.
- Se utiliza clasificador Naive Bayes ComplementNB con alpha = 0 y alpha = 1.0 (valor por defecto), para ver el efecto de la suavización

In [297]:
# Vectorización TF-IDF del punto 2.2
tfidfvect = TfidfVectorizer(min_df=(0.1/100),max_df=0.9,strip_accents='ascii')
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [298]:
# Clasificador: Naive Bayes ComplementNB con alpha=0
clf = ComplementNB(alpha=0)
clf.fit(X_train, y_train)
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)
score = f1_score(y_test, y_pred, average='macro')
score

C:\Users\lsaraco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
C:\Users\lsaraco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


0.6536621880859208

In [299]:
# Almacenamiento y muestra de métricas
df_metricas = almacenar_metricas(
        clasificador="ComplementNB con alpha=0",
        vectorizador="TF-IDF con min_df=0.1%, max_df=90% y strip_accents='ascii'",
        f1_score=score,
        existing_df=df_metricas
        )

display(df_metricas)

,Clasificador,Vectorizador,F1-score
0,MultinomialNB,TF-IDF con parámetros por defecto,0.585435
1,MultinomialNB,"TF-IDF con min_df=0.1%, max_df=90% y strip_acc...",0.614548
2,ComplementNB con alpha=0,"TF-IDF con min_df=0.1%, max_df=90% y strip_acc...",0.653662


Con alpha=1

In [300]:
# Clasificador: Naive Bayes ComplementNB con alpha=0
clf = ComplementNB(alpha=1)
clf.fit(X_train, y_train)
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)
score = f1_score(y_test, y_pred, average='macro')
score

0.6603254297989471

In [301]:
# Almacenamiento y muestra de métricas
df_metricas = almacenar_metricas(
        clasificador="ComplementNB con alpha=1",
        vectorizador="TF-IDF con min_df=0.1%, max_df=90% y strip_accents='ascii'",
        f1_score=score,
        existing_df=df_metricas
        )

display(df_metricas)

,Clasificador,Vectorizador,F1-score
0,MultinomialNB,TF-IDF con parámetros por defecto,0.585435
1,MultinomialNB,"TF-IDF con min_df=0.1%, max_df=90% y strip_acc...",0.614548
2,ComplementNB con alpha=0,"TF-IDF con min_df=0.1%, max_df=90% y strip_acc...",0.653662
3,ComplementNB con alpha=1,"TF-IDF con min_df=0.1%, max_df=90% y strip_acc...",0.660325


## **3**. Transponer la matriz documento-término. 

De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [302]:
# Se toma el conjunto de entrenamiento como la matriz documento-terminos, ya que es bastante grande (11314 documentos)
doc_term_matrix = X_train
term_doc_matrix = X_train.T
term_doc_matrix

<9349x11314 sparse matrix of type '<class 'numpy.float64'>'
	with 923727 stored elements in Compressed Sparse Column format>

Para la vectirización se usará el vectorizador del punto 2.2 (tfidf con parámetros min_df, max_df y strip_accents)

In [303]:
tfidfvect = TfidfVectorizer(min_df=(0.1/100),max_df=0.9,strip_accents='ascii')
tfidfvect.fit_transform(newsgroups_train.data)
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [304]:
# Lista de palabras con las que se estudiará su similaridad:
words_list = [
                "car",
                "radio",
                "health",
                "transistor",
                "president"
              ]

for w in words_list:
    print(f"La palabra '{w}' se encuentra en la fila {tfidfvect.vocabulary_[w]} de la variable term_doc_matrix.")


La palabra 'car' se encuentra en la fila 1679 de la variable term_doc_matrix.
La palabra 'radio' se encuentra en la fila 6830 de la variable term_doc_matrix.
La palabra 'health' se encuentra en la fila 4109 de la variable term_doc_matrix.
La palabra 'transistor' se encuentra en la fila 8604 de la variable term_doc_matrix.
La palabra 'president' se encuentra en la fila 6545 de la variable term_doc_matrix.


In [305]:
# Obtengo 5 más similares de cada palabra
for w in words_list:
    print(f"** Las 5 palabras más similares a '{w}' según similitud coseno son:")
    cossim = cosine_similarity(term_doc_matrix[tfidfvect.vocabulary_[w]], term_doc_matrix)[0]
    words_sim_idx = np.argsort(cossim)[::-1][1:(5+1)]
    for i in words_sim_idx:
        print(f"\t - {idx2word[i]}")

** Las 5 palabras más similares a 'car' según similitud coseno son:
	 - cars
	 - dealer
	 - civic
	 - owner
	 - the
** Las 5 palabras más similares a 'radio' según similitud coseno son:
	 - shack
	 - amateur
	 - ham
	 - tuned
	 - handbook
** Las 5 palabras más similares a 'health' según similitud coseno son:
	 - care
	 - insurance
	 - medical
	 - medicine
	 - underlying
** Las 5 palabras más similares a 'transistor' según similitud coseno son:
	 - diode
	 - flip
	 - thermal
	 - dense
	 - fuse
** Las 5 palabras más similares a 'president' según similitud coseno son:
	 - secretary
	 - clinton
	 - edt
	 - vice
	 - meets


### Conclusiones del punto 3:

- Si bien algunas de las palabras similares a la palabra principal no tienen tanto sentido, la mayoría de ellas sí, o pertenecen al mismo contexto.
- Por ejemplo, para el caso de 'car', tiene sentido que la más similar sea 'cars'. 'Owner' también tiene sentido, ya que seguramente la palabra 'car' está acompañada de 'owner' en varios documentos.
- En el caso de 'radio', su relación más fuerte es con 'shack', probablemente porque en varios documentos se hable de 'radio shack', la tienda estadounidense.
- Tomando otro caso, por ejemplo 'transistor', vemos que los términos similares pertenecen todos al ámbito de la electrónica.